In [11]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import mysql.connector
from datetime import datetime, timedelta

In [12]:
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'abcd1234',
    'database': 'iitr',
}
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

In [13]:
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()


In [14]:


query = "SELECT * FROM stockdata2024"

# Execute the query and load data into a pandas DataFrame
dataframe2 = pd.read_sql(query, connection)

# Close the database connection
connection.close()




/var/folders/vj/lr0smv8s7cjfn1_wjtdkj1c80000gn/T/ipykernel_96158/2465077945.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe2 = pd.read_sql(query, connection)


In [15]:
import numpy as np
dataframe2 = dataframe2.replace('-', np.nan)

In [16]:
dataframe2['DATE1'] = pd.to_datetime(dataframe2['DATE1'])

# Extract distinct dates from the 'date' column
distinct_dates = dataframe2['DATE1'].drop_duplicates()

# Format the dates as "DDMMYYYY"
dates2 = distinct_dates.dt.strftime('%d%m%Y').tolist()

# Display the list of formatted dates
print(dates2)
print(len(dates2))

['01012024', '02012024', '03012024', '04012024', '05012024', '08012024', '09012024', '10012024', '11012024', '12012024', '15012024', '16012024', '17012024', '18012024', '19012024', '20012024', '23012024', '24012024', '25012024', '29012024', '30012024', '31012024', '01022024', '02022024', '05022024', '06022024', '07022024', '08022024', '09022024', '12022024', '13022024', '14022024', '15022024', '16022024', '19022024', '20022024', '21022024', '22022024', '23022024', '26022024', '27022024', '28022024', '29022024', '01032024', '04032024', '05032024', '06032024', '07032024', '11032024', '12032024', '13032024', '14032024', '15032024', '18032024', '19032024', '20032024', '21032024', '22032024', '26032024', '27032024', '28032024', '01042024', '02042024', '03042024', '04042024', '05042024', '08042024', '09042024', '10042024', '12042024', '15042024', '16042024', '18042024', '19042024', '22042024', '23042024', '24042024', '25042024', '26042024', '29042024', '30042024', '02052024', '03052024', '06

In [17]:
df_filtered = dataframe2[~dataframe2['SYMBOL'].str.contains('ETF', na=False)]

In [18]:
dataframe2=df_filtered

In [20]:
final_ans6 = pd.DataFrame()
final_ans7 = pd.DataFrame()

for i in range(0,len(dates2) - 3):
    # Select 4 consecutive dates
    selected_dates = dates2[i:i + 4]
    # Filter the DataFrame to include only the selected dates
    selected_dates_datetime = [pd.to_datetime(date, format='%d%m%Y') for date in selected_dates]
    
    # Filter the DataFrame
    df= dataframe2[dataframe2['DATE1'].isin(selected_dates_datetime)]
    ans6,ans7= find_pattern(df)
    
    final_ans6 = pd.concat([final_ans6, ans6], ignore_index=True).drop_duplicates()
    final_ans7 = pd.concat([final_ans7, ans7], ignore_index=True).drop_duplicates()

# Write to CSV
final_ans6.to_csv('buy_6cond2024.csv', index=False)
final_ans7.to_csv('buy_7cond2024.csv', index=False)


/var/folders/vj/lr0smv8s7cjfn1_wjtdkj1c80000gn/T/ipykernel_96158/556011135.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.071942446043018' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i,'body'] = (100 * np.abs(df['CLOSE_PRICE'][i] - df['OPEN_PRICE'][i]) / (df['HIGH_PRICE'][i] - df['LOW_PRICE'][i]))
/var/folders/vj/lr0smv8s7cjfn1_wjtdkj1c80000gn/T/ipykernel_96158/556011135.py:23: RuntimeWarning: invalid value encountered in scalar divide
  df.at[i,'body'] = (100 * np.abs(df['CLOSE_PRICE'][i] - df['OPEN_PRICE'][i]) / (df['HIGH_PRICE'][i] - df['LOW_PRICE'][i]))
/var/folders/vj/lr0smv8s7cjfn1_wjtdkj1c80000gn/T/ipykernel_96158/556011135.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '59.78260869565225' has dtype incompatible with int64, please explicitly cast to a 

In [16]:
final_ans7.to_csv('check_filtered_stocks_7cond2024.csv', index=False)

In [19]:
def find_pattern(daf):
    df = daf.reset_index(drop=True).copy()
    required_columns = ['SYMBOL', 'CLOSE_PRICE', 'PREV_CLOSE', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'TTL_TRD_QNTY', 'DELIV_PER']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise KeyError(f"Missing columns: {', '.join(missing_columns)}")
    df['same script'] = 0
    for i in range(2, len(df)):
        if df['SYMBOL'][i] == df['SYMBOL'][i-1] == df['SYMBOL'][i-2]:
             df.at[i, 'same script'] = 1
             
    df['%price change'] = None
    for i in range(0, len(df)):
        df.at[i, '%price change'] = 100*(((df['CLOSE_PRICE'][i])-(df['PREV_CLOSE'][i]))/df['PREV_CLOSE'][i])

    df['%3 consecutive chng >2.5'] = 0
    for i in range(2, len(df)):
        if df['%price change'][i]>2.5 and df['%price change'][i-1]>2.5 and df['%price change'][i-2]>2.5:
            df.at[i, '%3 consecutive chng >2.5'] = 1
 
    df['body'] = 0
    for i in range(0, len(df)):
        df.at[i,'body'] = (100 * np.abs(df['CLOSE_PRICE'][i] - df['OPEN_PRICE'][i]) / (df['HIGH_PRICE'][i] - df['LOW_PRICE'][i]))
    
    df['body>65%'] = 0
    for i in range(2, len(df)):
        if df['body'][i]>65 and df['body'][i-1]>65 and df['body'][i-2]>65:
            df.at[i, 'body>65%'] = 1

    df['prev close<today close'] = 0
    for i in range(0, len(df)):
        if df['PREV_CLOSE'][i]<df['CLOSE_PRICE'][i]:
            df.at[i, 'prev close<today close'] = 1
    
    df['prev open<today open'] = 0
    for i in range(1, len(df)):
        if df['OPEN_PRICE'][i-1]<df['OPEN_PRICE'][i]:
            df.at[i, 'prev open<today open'] = 1

    df['Prev Day Vol< Today Vol']=0
    for i in range(2, len(df)):
        if df['TTL_TRD_QNTY'][i-2]<df['TTL_TRD_QNTY'][i-1] and df['TTL_TRD_QNTY'][i-1]<df['TTL_TRD_QNTY'][i]:
            df.at[i, 'Prev Day Vol< Today Vol'] = 1
    df['DELIV_PER'] = pd.to_numeric(df['DELIV_PER'], errors='coerce')

    df['Prev day del<Today Del'] = 0
    for i in range(1, len(df)):
    # Check if both previous and current values are not NaN
        if not pd.isna(df.at[i-1, 'DELIV_PER']) and not pd.isna(df.at[i, 'DELIV_PER']):
        # Compare previous day's DELIV_PER with today's DELIV_PER
            if df.at[i-1, 'DELIV_PER'] < df.at[i, 'DELIV_PER']:
                df.at[i, 'Prev day del<Today Del'] = 1
    
    df['sum']=0
    for i in range(2, len(df)):
        df.at[i, 'sum'] = df['same script'][i]+df['%3 consecutive chng >2.5'][i]+df['prev close<today close'][i]+df['prev open<today open'][i]+ df['body>65%'][i]+df['Prev Day Vol< Today Vol'][i]+df['Prev day del<Today Del'][i]
    res7=df[df['sum'] >= 7]

    df['sum_without_del']=0
    for i in range(2, len(df)):
        df.at[i, 'sum_without_del'] = df['same script'][i]+df['%3 consecutive chng >2.5'][i]+df['prev close<today close'][i]+df['prev open<today open'][i]+ df['body>65%'][i]+df['Prev Day Vol< Today Vol'][i]
    res6=(df[df['sum_without_del']>=6])
    return res6,res7

